In [86]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 


In [87]:
train__set_features = pd.read_csv("training_set_features.csv")
test = pd.read_csv("test_set_features.csv")
train__set_features.shape

train__set_labels = pd.read_csv("training_set_labels.csv")

y_h1n1 = train__set_labels['h1n1_vaccine']
y_h1n1

y_seasonal = train__set_labels['seasonal_vaccine']

train__set_features.shape

train__set_labels['h1n1_vaccine'].value_counts()

0    21033
1     5674
Name: h1n1_vaccine, dtype: int64

In [88]:
y_seasonal.shape

(26707,)

In [89]:
train = pd.concat([train__set_features, y_h1n1, y_seasonal], axis=1)
train.columns



Index(['respondent_id', 'h1n1_concern', 'h1n1_knowledge',
       'behavioral_antiviral_meds', 'behavioral_avoidance',
       'behavioral_face_mask', 'behavioral_wash_hands',
       'behavioral_large_gatherings', 'behavioral_outside_home',
       'behavioral_touch_face', 'doctor_recc_h1n1', 'doctor_recc_seasonal',
       'chronic_med_condition', 'child_under_6_months', 'health_worker',
       'health_insurance', 'opinion_h1n1_vacc_effective', 'opinion_h1n1_risk',
       'opinion_h1n1_sick_from_vacc', 'opinion_seas_vacc_effective',
       'opinion_seas_risk', 'opinion_seas_sick_from_vacc', 'age_group',
       'education', 'race', 'sex', 'income_poverty', 'marital_status',
       'rent_or_own', 'employment_status', 'hhs_geo_region', 'census_msa',
       'household_adults', 'household_children', 'employment_industry',
       'employment_occupation', 'h1n1_vaccine', 'seasonal_vaccine'],
      dtype='object')

On va gerer les valeurs manquantes et les valeurs categoriels 

In [90]:
train.select_dtypes(include='object').isna().sum()
# train = train.drop(['employment_industry'], axis=1)
# la moitié de employement _industry et _occupation est manquante 


age_group                    0
education                 1407
race                         0
sex                          0
income_poverty            4423
marital_status            1408
rent_or_own               2042
employment_status         1463
hhs_geo_region               0
census_msa                   0
employment_industry      13330
employment_occupation    13470
dtype: int64

In [91]:
train = train.drop(["employment_industry", "employment_occupation"], axis=1)
test = test.drop(["employment_industry", "employment_occupation"], axis=1)

col = train.select_dtypes(exclude=(np.number)).columns

train = pd.get_dummies(train, columns=col)
test = pd.get_dummies(test, columns=col)

for col in train.columns: 
    if not(col in test.columns):
        test[col] = pd.Series(np.zeros(test.shape[0]), name=col)
test.shape



(26708, 64)

In [92]:
train.shape

(26707, 64)

In [93]:
from sklearn.neighbors import KNeighborsRegressor
def manage_na(X,na_col): 
    X = X.copy()
    numerical = X.select_dtypes(np.number)
    non_na_columns = numerical.loc[:,numerical.isna().sum() == 0].columns
    X_train = numerical.loc[numerical[na_col].isna() == False,non_na_columns]
    y_train = numerical.loc[numerical[na_col].isna() == False,na_col]
    X_na = numerical.loc[numerical[na_col].isna() == True, non_na_columns]
    knn = KNeighborsRegressor()
    knn.fit(X_train, y_train)
    y_na = knn.predict(X_na)
    X.loc[X[na_col].isna() == True, na_col] = y_na
    
    return X 

In [94]:
cond = train.corr()['seasonal_vaccine'].apply(abs).sort_values() > 0.1

nulls = train[cond.index[cond]].isna().sum() > 0

for col in nulls[nulls == True].index:
    train = manage_na(train, col)




In [95]:
nulls = train.isna().sum().sort_values() > 0
nulls.sort_values()


respondent_id                  False
education_< 12 Years           False
age_group_65+ Years            False
h1n1_concern                   False
h1n1_knowledge                 False
                               ...  
behavioral_large_gatherings     True
behavioral_avoidance            True
household_adults                True
opinion_h1n1_sick_from_vacc     True
child_under_6_months            True
Length: 64, dtype: bool

In [96]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV, train_test_split

y = train["seasonal_vaccine"]

X = train.drop("seasonal_vaccine", axis=1)


params = {
    "n_estimators": np.arange(10, 15,1), 
    "max_depth": np.arange(10, 15, 1),
    "min_samples_leaf": np.arange(5, 10, 1), 
}
grid = GridSearchCV(RandomForestRegressor(), params, cv=5)

nulls = X.isna().sum() > 0 

for col in nulls[nulls == True].index:
    X = manage_na(X, col)

X_train, X_test, y_train, y_test = train_test_split(X, y,  test_size=0.2, random_state=32,)

grid.fit(X_train, y_train)
grid.best_params_





{'max_depth': 10, 'min_samples_leaf': 9, 'n_estimators': 13}

In [97]:
rf = RandomForestRegressor(max_depth=13,min_samples_leaf=7, n_estimators=14)

rf.fit(X_train, y_train)

y_seasonal = rf.predict(X_test)
y_seasonal

rf.score(X, y)

y_seasonal


array([0.04271919, 0.75023047, 0.98218528, ..., 0.93165727, 0.07660079,
       0.90812398])

In [98]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, y_seasonal)

ValueError: Classification metrics can't handle a mix of binary and continuous targets

In [ ]:
sub_format = pd.read_csv("submission_format.csv")



0.7